# Презентация KNN
---

## Слайд 1

Вступительный слайд

## Слайд 2. Тема, цель, гипотеза

Доброе утро !

Сегодня мы рады представить результаты по работе над проектом **Hyperbolic kNN**.

На курсе была поставлена задача: проверить, как меняется качество kNN, если заменить стандартную евклидову геометрию и косинусную близость на **гиперболическое пространство** и гиперболическую метрику.

Мотивация задачи следующая: гиперболические пространства хорошо описывают **иерархии** и структуры вида дерева. В покупках часто можно видеть иерархические шаблоны: категория → подкатегория → товар, и пользовательские предпочтения тоже могут иметь подобную структуру.

Из мотивации была выдвинута гипотеза: если kNN измеряет похожесть в геометрии, которая лучше соответствует структуре данных, то качество рекомендаций может улучшиться.

---

## Слайд 3. Данные и постановка задачи

Итак, мы решаем задачу **next-basket recommendation**: у каждого пользователя есть история покупок, и по этой истории нужно предсказать товары следующей корзины.

Используем датасет **Ta-Feng** — транзакции супермаркета. Транзакции агрегируем в **дневные корзины**: для каждого пользователя и дня получаем множество уникальных товаров, купленных в этот день.

Так для каждого пользователя получается временная последовательность корзин.

Оставляем только пользователей с хотя бы тремя корзинами, чтобы корректно построить train, validation и test.

---

## Слайд 4. Эксперименты и метрики

Важно недопустить учтечки данных в будущее, поэтому мы используем **time-based split per user**: сортируем корзины по времени и делим так: train — все корзины, кроме двух последних; validation — предпоследняя; test — последняя. Любые матрицы взаимодействий и статистики строятся **только по train**.

Метрики:

- **Recall@K** — доля истинной корзины, попавшая в топ-K рекомендаций,
- **NDCG@K** — качество ранжирования, то есть насколько высоко стоят релевантные товары,
- **Coverage@K** — доля каталога, которая появляется в рекомендациях по всем пользователям.

Отчётные значения K: **10 и 100**, чтобы видеть поведение на небольшой выборке и на более длинном списке.

---

## Слайд 5. Бейзлайны

Чтобы честно оценить гиперболический kNN, мы построили ряд базовых моделей:

1. **TopPop** — рекомендации самых популярных товаров в train. Это нижняя граница и проверка силы популярности в датасете.
2. **UserKNN (cosine)** — основной baseline. Именно его мы затем гиперболизируем.
3. **TIFU-KNN (simple)** — time-aware kNN: строим два профиля: IU (item usage) — бинарный “встречался ли товар”, PIF (Products Interaction Frequency) — частоты с временным затуханием по корзинам. Похожесть — смесь косинусов PIF и IU с коэффициентом alpha.
4. **SVD (TruncatedSVD)** — быстрый baseline, основанный на матричной факторизации.
5. **ItemKNN (cosine)** — item-based вариант kNN. Помогает более широко посмотреть на модели KNN.

Гиперпараметры тюнили на подмножестве пользователей по **NDCG@10**, затем фиксировали лучшие и оценивали на полных валидации и тесте.

---

## Слайд 6. Гиперболическое пространство

Теперь про гиперболические варианты. Мы реализовали **Hyperbolic ItemKNN**, **Hyperbolic UserKNN** и **Hyperbolic TIFU-KNN** в модели Пуанкаре: считаем гиперболическое расстояние и преобразуем его в похожесть. Пробовали три преобразования: Экспоненциальное (exp(−d)), Инверсионное (1/(1+d)) и гауссовское.

В текущей реализации гиперболический KNN **не превзошёл** бейзлайны с евклидовой геометрией по NDCG/Recall. Это важный результат, который говорит нам, что замена геометрии не гарантирует улучшений. Мы также видим, что выбор функции similarity сильно влияет на coverage: некоторые варианты расширяют покрытие каталога, но качество падает.

---

## Слайд 7. Гиперболические эмбеддинги

В методе poincare_loss реализуется контрастивная потеря с негативным сэмплированием, аналогичная Noise Contrastive Estimation (NCE) или InfoNCE:
Для каждой положительной пары (i, j) и K негативных элементов (negs):
$$\ell = -\log\left[ \frac{\exp(-d(i,j))}{\exp(-d(i,j)) + \sum_k \exp(-d(i,k))} \right]$$

Есть важный нюанс: эмбеддинги должны лежать внутри шара Пуанкаре радиуса 1, т.е. их норма должна быть строго меньше 1.
Поэтому после каждого шага оптимизатора применяется проекция на шар:
$$
\tilde{x} = 
\begin{cases} 
x & \text{если } \|x\| < 1 - \epsilon \\
\frac{x}{\|x\|} \cdot (1 - \epsilon) & \text{если } \|x\| \geq 1 - \epsilon
\end{cases}

$$
где:
- ||x|| — евклидова норма вектора (x) (L2-норма),
- $\epsilon$ — маленькая положительная константа (в коде `EPS = 1e-5`), введённая для численной стабильности и чтобы избежать выхода на границу шара (расстояние Пуанкаре определено только строго внутри шара).

Мы минимизируем среднюю по всем положительным парам контрастивную потерю:
$$L = \frac{1}{|E|} \sum_{(i,j) \in E} -\log\left[ \frac{\exp(-d(i,j))}{\exp(-d(i,j)) + \sum_k \exp(-d(i,k))} \right]$$


---

## Слайд 8. Результаты экспериментов

По результатам на **TEST** лидеры по качеству — **UserKNN** и **TIFU-KNN**.

Если смотреть на ранжирование небольшого числа товаров, **UserKNN** лучший по NDCG@10, у TIFU эта метрика чуть ниже. Если смотреть на Recall, TIFU немного лучше. На K=100 сохраняется та же картина: UserKNN лучше по NDCG@100, TIFU — чуть лучше по Recall@100. Разница небольшая, но стабильная.

Сравнение TopPop и SVD показывает, что в этих данных большое значение имеет популярность: популярностная модель на test сопоставима с SVD по NDCG@10 и лучше при оценки топ-100 товаров в корзине.

Теперь немного про **coverage**.
Популярностная модель почти не покрывает каталог, так как выдаёт фиксированный набор. SVD увеличивает coverage, но не сильно. UserKNN и TIFU дают существенно более высокий coverage при хорошем качестве — порядка половины каталога при K=100. ItemKNN почти покрывает весь каталог: coverage около **0.91** при K=100, но при этом сильно уступает по NDCG и Recall. Это демонстрирует компромисс между разнообразием и точностью.

---

## Слайд 9. Заключение

В ходе проекта мы построили воспроизводимый протокол next-basket рекомендаций на Ta-Feng с корректным time-based split и набором сильных бейзлайнов, включая метрики Recall@K, NDCG@K и Coverage@K для K = 10 и 100.

По качеству на тесте лучшими оказались **UserKNN** и **TIFU-KNN**: UserKNN немного выигрывает по NDCG, а TIFU — по Recall и coverage, что отражает ожидаемый компромисс между ранжированием верхушки и более широким покрытием релевантных товаров.

Гиперболические варианты в текущей реализации не превзошли евклидовые бейзлайны, что показывает: простая замена метрики сама по себе не гарантирует улучшений. Основное направление дальнейшей работы — переход к **обучаемым гиперболическим эмбеддингам** и более тщательная настройка функции similarity/параметров геометрии, чтобы гиперболическое пространство реально отражало структуру данных.


## Слайд 10.

Спасибо за внимание ! 
Будем рады ответить на вопросы !
